# Derivatives, Integrals, and Taylor Series

List of contributors: Joshua Wylie

Welcome back to everyone who has completed other modules and a double welcome to those who are using this as their first module in our series! The goal of this notebook is to highlight derivative, integral, and Taylor Series concepts along with numerical implementations. If at any point you see code you're unfamiliar with (without explanation), [please see the appropriate notebooks in our GitHub](https://github.com/ascsn/online-guides/tree/main) which we'll do our best to reference. Before we throw you into the deep end, we recommend familiarizing yourself with the following topics and packages:
 - INSERT TOPICS (probably things like using numpy, python, etc.)

---

In [ ]:
import numpy as np
import pandas as pd

# We can import matplotlib which has many nice and simple features, particularly for simple plot types!
import matplotlib.pyplot as plt

# Derivatives

If you're participating in our course, you've hopefully already found a deeper appreciation for derivatives but if not, that's what we're hoping this notebook will help do too! As a brief recap of the idea of derivatives, let's start with some motivation by considering the motion of a person walking back and forth. We might say their path follows something like below:

 1. The person walks at a pace consistent with time, maybe 1 m/s
 2. The person pauses and waits for a period of time
 3. The person realizes the missed something and has to run back accelerating as they go
 4. The person then starts walking back at their original pace

or in an equation form
\begin{equation}
\text{position}(t) = 
\begin{cases} 
t & 0 \leq t \leq t_1 \\
t_1 & t_1 < t \leq t_2 \\
t_1 - \frac{(t - t_2)^2}{2} & t_2 <  t \leq t_3 \\
t_1 - \frac{(t_3 - t_2)^2}{2} + (t - t_3) & t_3 < t
\end{cases}
\end{equation}
which we can also define in python.

In [ ]:
# Define time intervals
t1, t2, t3, t4 = 2, 5, 8, 12
total_time = t4
total_time_steps = 200

# Define time array
time = np.linspace(0, total_time, total_time_steps)

# Piecewise function for position
def position(t):
    if t <= t1:
        return t # The person walks at a pace consistent with time, maybe 1 m/s
    elif t <= t2:
        return t1 # The person pauses and waits for a period of time
    elif t <= t3:
        return t1 - (t - t2)**2 / 2 # The person realizes the missed something and has to run back accelerating as they go
    else:
        return t1 - (t3 - t2)**2 / 2 + (t - t3) # The person then starts walking back at their original pace

# Calculate positions
positions = np.array([position(t) for t in time])

This might be a bit tough to visualize without drawing things out, so let's also do that in our notebook now! We'll get a little fancy with the visualization and use something called ```plotly``` which is like ```matplotlib``` in that it is a plotting library but it includes some extra features that are nice for interactive figures.

In [ ]:
# For our plots, we'll make use of plotly when we want to make things fancy!
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.io as pio

In [ ]:
# Create a figure for the position vs time plot
fig = go.Figure()

# Add the position vs time trace
fig.add_trace(go.Scatter(x=time, y=positions, mode='lines', name='Position'))
fig.add_trace(go.Scatter(x=[time[0]], y=[positions[0]], mode='markers', marker=dict(color='blue', size=10), name='Current Position'))

# Create frames for the slider
frames = [go.Frame(data=[
    go.Scatter(x=[time[i]], y=[positions[i]], mode='markers', marker=dict(color='blue', size=10), name='Current Position')
], traces=[1], name=str(i)) for i in range(len(time))]

# Add frames to the figure
fig.frames = frames
frames_to_skip = 10 # Only show 1 out of 10 frames

# Define animation settings for play/pause buttons
animation_settings = dict(frame=dict(duration=20, redraw=True), fromcurrent=True)

# Update layout with buttons and sliders
fig.update_layout(
    updatemenus=[dict(
        type="buttons",  # Define the type of control
        showactive=False,  # Do not show active state
        buttons=[
            dict(label="Play",
                 method="animate",  # Use the animate method
                 args=[None, animation_settings]),  # Settings for the animation
            dict(label="Pause",
                 method="animate",  # Use the animate method
                 args=[[None], dict(frame=dict(duration=0, redraw=False), mode="immediate", transition=dict(duration=0))])
        ]
    )],
    sliders=[{
        'steps': [{'args': [[str(i)], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate'}],
                   'label': str(round(time[i], 2)), 'method': 'animate'} 
                   for i in range(len(time)) if i % frames_to_skip == 0],  # Define each step of the slider in a list comprehension
        'transition': {'duration': 0},  # No transition duration between steps
        'x': 0.1,  # X position of the slider
        'y': -0.1,  # Y position of the slider
        'currentvalue': {'font': {'size': 20}, 'prefix': 'Time: ', 'visible': True, 'xanchor': 'center'},  # Current value settings
        'len': 0.9  # Length of the slider
    }]
)

# Update axes titles and ranges
fig.update_xaxes(title_text="Time (s)", range=[0, total_time])  # X-axis settings
fig.update_yaxes(title_text="Position (units)", range=[min(positions) - 1, max(positions) + 1])  # Y-axis settings

# Set title and layout dimensions
fig.update_layout(title_text="Position vs Time Animation with Slider", height=600, width=800, showlegend=False)

# Display the figure
pio.show(fig)

Excellent! Now we have a nice visualization of the position of the particle with time, so we can get back to our derivative discussion.

## Macroscopic Rate of Change

Let's say we want to know how fast the person moved from each point to the other. In other words we can always take a basic definition of a rate
\begin{equation}
{\rm position\;rate} = \frac{\rm position\;change}{\rm time\;change}=\frac{\Delta x}{\Delta t} = \frac{x_f - x_i}{t_f - t_i}
\end{equation}
to get our rate of change of position or a velocity. In the cell below, we show the different average rates of change for each segment of the piecewise position plot.

In [ ]:
# Define our rate calculation function
def get_rate(function_, variable_, start_index, end_index):
    rate = (function_[end_index] - function_[start_index]) / (variable_[end_index] - variable_[start_index])
    return np.repeat(rate, 2) # Repeat each element twice in order to get double the dimensions of the indices e.g. setting the rate start and end points

# Calculate the velocity over our different indices
# We will make a 2D numpy array where each row contains the start and end time
desired_time_segments = np.array([[0, t1], [t1, t2], [t2, t3], [t3, t4]])
print('Desired time points selected =', desired_time_segments.flatten())

def find_nearest_index(array, value):
    # Check if we have a value or a numpy array
    if isinstance(value, np.ndarray):
        original_shape = value.shape
        value = value.flatten()
        idx = np.array([(np.abs(array - val)).argmin() for val in value])
        idx = idx.reshape(original_shape)
    else: # Otherwise assume we're given a float
        array = np.asarray(array)
        idx = (np.abs(array - value)).argmin()
    return idx

# Get indices where time segments match the time array we constructed
segment_indices = find_nearest_index(time, desired_time_segments)

# Let's see what time values were selected
start_times = time[segment_indices[:,0]]
end_times = time[segment_indices[:,1]]
print('start times =', start_times)
print('end times =', end_times)

# Now we can select the corresponding positions
start_positions = positions[segment_indices[:,0]]
end_positions = positions[segment_indices[:,1]]
print('start positions =', start_positions)
print('end positions =', end_positions)

# And we can then calculate our rates
position_rates = get_rate(positions, time, segment_indices[:, 0], segment_indices[:, 1])
print('position rates (average velocity) =', position_rates)

# See what position points we will call by flattening the segment 2D array
print('All positions points selected =', positions[segment_indices.flatten()])

# All of the above is not very efficient, but we tried to do it for some hopeful clarity.
# If you'd like to practice your numpy skills, you can try new ways to program the above!

In [ ]:
# Let's plot our figures
# Create a figure for the position vs time plot
fig = make_subplots(rows=1, cols=1, subplot_titles=("Position vs Time with Average Rates of Change",))

# Add the position vs time trace
fig.add_trace(go.Scatter(x=time, y=positions, mode='lines', name='Position'), row=1, col=1)

# Add the rate of position vs time trace
fig.add_trace(go.Scatter(x=time[segment_indices.flatten()], y=position_rates, mode='lines', name='Position Rate'), row=1, col=1)

# Update layout
fig.update_xaxes(title_text="Time (s)", range=[0, total_time], row=1, col=1)
fig.update_yaxes(title_text="Position (units)", range=[min(positions) - 1, max(positions) + 1], row=1, col=1)
fig.update_layout(title_text="Position vs Time with Average Rates of Change", height=600, width=800, showlegend=False)

# Display the figure
pio.show(fig)

On the surface this seems good enough to understand where the person was moving or standing still; however, if we remember the problem, there was a point where the person was accelerating and our change in position was not a linear change. To illustrate this, we take the previous figure of the rates of change and split each segement into two.

In [ ]:
# We will make a 2D numpy array again but this time include another row where we divide the time in half
intermediate_time = t2 + (t3 - t2)/2
desired_time_segments = np.array([[0, t1], [t1, t2], [t2, intermediate_time], [intermediate_time, t3], [t3, t4]])
print('Desired time points selected =', desired_time_segments.flatten())

# Get indices where time segments match the time array we constructed
segment_indices = find_nearest_index(time, desired_time_segments)

# And we can then calculate our rates
position_rates = get_rate(positions, time, segment_indices[:, 0], segment_indices[:, 1])

# Let's plot our figures
# Create a figure for the position vs time plot
fig = make_subplots(rows=1, cols=1, subplot_titles=("Position vs Time with Average Rates of Change",))

# Add the position vs time trace
fig.add_trace(go.Scatter(x=time, y=positions, mode='lines', name='Position'), row=1, col=1)

# Add the rate of position vs time trace
fig.add_trace(go.Scatter(x=time[segment_indices.flatten()], y=position_rates, mode='lines', name='Position Rate'), row=1, col=1)

# Update layout
fig.update_xaxes(title_text="Time (s)", range=[0, total_time], row=1, col=1)
fig.update_yaxes(title_text="Position (units)", range=[min(positions) - 1, max(positions) + 1], row=1, col=1)
fig.update_layout(title_text="Position vs Time with Average Rates of Change", height=600, width=800, showlegend=False)

# Display the figure
pio.show(fig)

Okay, maybe we were right to say that our rate of change in an area was inconsistent with the others. We see in the acceleration region that when we divide up the segments the velocity value has changed unlike the other constant rate segments. To make this a bit more insightful, we can add in a slider which changes the number of rate segments we are dividing each position vs time event by.

In [ ]:
# We will make a 2D numpy array again but this time include another row where we divide the time in half

# We need to create a series of frames with different n divisions e.g. total_time / n
max_discretization_points = 50
discretization_step_size = 1

# We define our original array of desired time segments ignoring the acceleration segment
desired_time_segments = np.array([[0, t1], [t1, t2], [t3, t4]], dtype=float)

Dt = t3 - t2
# We can define a list of desired time segments where we insert an array of intermediate points with different numbers of discretization points
list_of_desired_time_segments = [
    np.insert(desired_time_segments, 2, np.vstack([np.arange(t2, t3, Dt/n), np.arange(t2, t3, Dt/n) + Dt/n]).T, axis=0) for n in range(1, max_discretization_points, discretization_step_size)
    ]

# Get indices where time segments match the time array we constructed
list_of_segment_indices = [find_nearest_index(time, ts) for ts in list_of_desired_time_segments]

# And we can then calculate our rates
list_of_position_rates = [get_rate(positions, time, lsi[:,0], lsi[:,1]) for lsi in list_of_segment_indices]

# Let's plot our figures!
# Create a figure for the position vs time plot
fig = make_subplots(rows=1, cols=1, subplot_titles=("Position vs Time with Average Rates of Change",))

# Add the position vs time trace
fig.add_trace(go.Scatter(x=time, y=positions, mode='lines', name='Position'), row=1, col=1)

# Add the rate of position vs time trace
fig.add_trace(go.Scatter(x=time[list_of_segment_indices[0]].flatten(), y=list_of_position_rates[0], mode='lines', name='Position Rate'), row=1, col=1)

# Create frames for the slider
frames = [go.Frame(data=[
    go.Scatter(x=time[list_of_segment_indices[i]].flatten(), y=list_of_position_rates[i], mode='lines', name='Position Rate')
], traces=[1], name=str(i)) for i in range(len(list_of_segment_indices))]


# Add frames to the figure
fig.frames = frames

# Define animation settings for play/pause buttons
animation_settings = dict(frame=dict(duration=20, redraw=True), fromcurrent=True)

# Update layout with buttons and sliders
fig.update_layout(
    updatemenus=[dict(
        type="buttons",  # Define the type of control
        showactive=False,  # Do not show active state
        buttons=[
            dict(label="Play",
                 method="animate",  # Use the animate method
                 args=[None, animation_settings]),  # Settings for the animation
            dict(label="Pause",
                 method="animate",  # Use the animate method
                 args=[[None], dict(frame=dict(duration=0, redraw=False), mode="immediate", transition=dict(duration=0))])
        ]
    )],
    sliders=[{
        'steps': [{'args': [[str(i)], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate'}],
                   'label': str(i), 'method': 'animate'} 
                   for i in range(len(list_of_segment_indices))],  # Define each step of the slider in a list comprehension
        'transition': {'duration': 0},  # No transition duration between steps
        'x': 0.1,  # X position of the slider
        'y': -0.1,  # Y position of the slider
        'currentvalue': {'font': {'size': 20}, 'prefix': 'Discretization points: ', 'visible': True, 'xanchor': 'center'},  # Current value settings
        'len': 0.9  # Length of the slider
    }]
)

# Update axes titles and ranges
fig.update_xaxes(title_text="Time (s)", range=[0, total_time])  # X-axis settings
fig.update_yaxes(title_text="Position (units)", range=[min(positions) - 1, max(positions) + 1])  # Y-axis settings
fig.update_layout(title_text="Position vs Time with Average Rates of Change", height=600, width=800, showlegend=False)

# Set title and layout dimensions
fig.update_layout(title_text="Position vs Time Animation with Slider", height=600, width=800, showlegend=False)

# Display the figure
pio.show(fig)

You might be thinking something like, "This seems like a tedious way to find the velocity at each point since we'd need to look at smaller and smaller segments" and you'd be entirely correct since this is where derivatives come in!

## Microscopic (infinitesmal) Rate of Change

If we have an analytic functional form of our position, we can calculate the derivative of the entire function
\begin{equation}
\text{velocity}(t) = \frac{d}{dt} \text{position}(t)=
\begin{cases} 
1 & 0 \leq t \leq t_1 \\
0 & t_1 < t \leq t_2 \\
t_2 - t & t_2 <  t \leq t_3 \\
1 & t_3 < t
\end{cases}
\end{equation}

We show this analytic form in the figure below.

In [ ]:
# Piecewise function for position
def velocity(t):
    if t <= t1:
        return 1 # The person walks at a pace consistent with time, maybe 1 m/s
    elif t <= t2:
        return 0 # The person pauses and waits for a period of time
    elif t <= t3:
        return t2 - t # The person realizes the missed something and has to run back accelerating as they go
    else:
        return 1 # The person then starts walking back at their original pace

# Calculate velocities
velocities = np.array([velocity(t) for t in time])

In [ ]:
# Create a figure for the position vs time plot
fig = go.Figure()

# Add the position vs time trace
fig.add_trace(go.Scatter(x=time, y=positions, mode='lines', name='Position'))
fig.add_trace(go.Scatter(x=[time[0]], y=[positions[0]], mode='markers', marker=dict(color='blue', size=10), name='Current Position'))

# Add the velocity vs time trace
fig.add_trace(go.Scatter(x=time, y=velocities, mode='lines', name='Velocity'))
fig.add_trace(go.Scatter(x=[time[0]], y=[velocities[0]], mode='markers', marker=dict(color='green', size=10), name='Current Velocity'))

# Create frames for the slider
frames = [go.Frame(data=[
    go.Scatter(x=time, y=positions, mode='lines', name='Position'),
    go.Scatter(x=[time[i]], y=[positions[i]], mode='markers', marker=dict(color='blue', size=10), name='Current Position'),
    go.Scatter(x=time, y=velocities, mode='lines', name='Velocity'),
    go.Scatter(x=[time[i]], y=[velocities[i]], mode='markers', marker=dict(color='green', size=10), name='Current Velocity')
], name=str(i)) for i in range(len(time))]

# Add frames to the figure
fig.frames = frames
frames_to_skip = 10  # Only show 1 out of 10 frames

# Define animation settings for play/pause buttons
animation_settings = dict(frame=dict(duration=20, redraw=True), fromcurrent=True)

# Update layout with buttons and sliders
fig.update_layout(
    updatemenus=[dict(
        type="buttons",  # Define the type of control
        showactive=False,  # Do not show active state
        buttons=[
            dict(label="Play",
                 method="animate",  # Use the animate method
                 args=[None, animation_settings]),  # Settings for the animation
            dict(label="Pause",
                 method="animate",  # Use the animate method
                 args=[[None], dict(frame=dict(duration=0, redraw=False), mode="immediate", transition=dict(duration=0))])
        ]
    )],
    sliders=[{
        'steps': [{'args': [[str(i)], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate'}],
                   'label': str(round(time[i], 2)), 'method': 'animate'}
                  for i in range(len(time)) if i % frames_to_skip == 0],  # Define each step of the slider in a list comprehension
        'transition': {'duration': 0},  # No transition duration between steps
        'x': 0.1,  # X position of the slider
        'y': -0.1,  # Y position of the slider
        'currentvalue': {'font': {'size': 20}, 'prefix': 'Time: ', 'visible': True, 'xanchor': 'center'},  # Current value settings
        'len': 0.9  # Length of the slider
    }]
)

# Update axes titles and ranges
fig.update_xaxes(title_text="Time (s)", range=[0, total_time])  # X-axis settings
fig.update_yaxes(title_text="Position and Velocity (units)", range=[min(min(positions), min(velocities)) - 1, max(max(positions), max(velocities)) + 1])  # Y-axis settings

# Set title and layout dimensions
fig.update_layout(title_text="Position vs Time and Velocity vs Time Animation with Slider", height=600, width=800, showlegend=True)

# Display the figure
pio.show(fig)


#### Concept Question 1: 
Check back to the previous case where we divide the acceleration region of the position into different segments, how does that compare to our analytic case?

<details>
  <summary>Click here for sample answer</summary>
  
  They appear to be the same! This is because a derivative takes the infinitesimal rate of change for a function, e.g. this is the same as setting $\Delta t \rightarrow \varepsilon$ where $\varepsilon$ is a small number approaching zero. This is what we are doing explicitly with our slider term where the segments approach smaller and smaller values of $\varepsilon$.
</details>

This is a calculus heavy version of our solution but we can also take a more geometric interpretation to get to the same destination.

## Tangent lines (Section needs to be finished!)

There is a geometrical interpretation

In [ ]:
# Calculate unit tangent and normal vectors
speed = np.sqrt(velocities**2)
mask = speed != 0  # Avoid divide by zero warning
tangent_x = np.ones_like(speed)
tangent_y = np.zeros_like(speed)
tangent_x[mask] = velocities[mask] / speed[mask]

normal_x = np.zeros_like(tangent_x)
normal_y = np.ones_like(tangent_y)

# Tangent and normal vectors' end coordinates
xend = positions + tangent_x
yend = np.zeros_like(positions) + tangent_y

xnoe = positions + normal_x
ynoe = np.zeros_like(positions) + normal_y

# Create the figure
fig = go.Figure()

# Add the position vs time trace
fig.add_trace(go.Scatter(x=time, y=positions, mode='lines', name='Position'))

# Add the velocity vs time trace
fig.add_trace(go.Scatter(x=time, y=velocities, mode='lines', name='Velocity'))

# Add the initial current position marker
fig.add_trace(go.Scatter(x=[time[0]], y=[positions[0]], mode='markers', marker=dict(color='blue', size=10), name='Current Position'))

# Add the initial current velocity marker
fig.add_trace(go.Scatter(x=[time[0]], y=[velocities[0]], mode='markers', marker=dict(color='green', size=10), name='Current Velocity'))

# Create frames for the slider
frames = []
for k in range(len(time)):
    t = time[k]
    pos = positions[k]
    vel = velocities[k]
    
    # Tangent line calculation
    tangent_x = np.linspace(t - 1, t + 1, 100)
    tangent_y = pos + vel * (tangent_x - t)
    
    frames.append(go.Frame(data=[
        go.Scatter(x=[time[k]], y=[positions[k]], mode='markers', marker=dict(color='blue', size=10), name='Current Position'),
        go.Scatter(x=[time[k]], y=[velocities[k]], mode='markers', marker=dict(color='green', size=10), name='Current Velocity'),
        go.Scatter(x=[time[k], time[k] + tangent_x[1]], y=[positions[k], tangent_y[1]], mode='lines', line=dict(color='red', dash='dash'), name='Tangent Line')
    ], name=str(k)))

frames_to_skip = 10  # Only show 1 out of 10 frames

# Define animation settings for play/pause buttons
animation_settings = dict(frame=dict(duration=20, redraw=True), fromcurrent=True)

# Update layout with buttons and sliders
fig.update_layout(
    updatemenus=[dict(
        type="buttons",  # Define the type of control
        showactive=False,  # Do not show active state
        buttons=[
            dict(label="Play",
                 method="animate",  # Use the animate method
                 args=[None, animation_settings]),  # Settings for the animation
            dict(label="Pause",
                 method="animate",  # Use the animate method
                 args=[[None], dict(frame=dict(duration=0, redraw=False), mode="immediate", transition=dict(duration=0))])
        ]
    )],
    sliders=[{
        'steps': [{'args': [[str(i)], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate'}],
                   'label': str(round(time[i], 2)), 'method': 'animate'}
                  for i in range(len(time)) if i % frames_to_skip == 0],  # Define each step of the slider in a list comprehension
        'transition': {'duration': 0},  # No transition duration between steps
        'x': 0.1,  # X position of the slider
        'y': -0.1,  # Y position of the slider
        'currentvalue': {'font': {'size': 20}, 'prefix': 'Time: ', 'visible': True, 'xanchor': 'center'},  # Current value settings
        'len': 0.9  # Length of the slider
    }]
)

# Update axes titles and ranges
fig.update_xaxes(title_text="Time (s)", range=[0, total_time])  # X-axis settings
fig.update_yaxes(title_text="Position and Velocity (units)", range=[min(positions) - 1, max(positions) + 1])  # Y-axis settings

# Set title and layout dimensions
fig.update_layout(title_text="Position and Velocity vs Time with Tangent Line Animation", height=600, width=800, showlegend=True)

# Add frames to the figure
fig.frames = frames

# Display the figure
pio.show(fig)


We see that the tangent line corresponds exactly to our derivative.

#### Concept Question 2: 

From the perspective of tangent lines, why is it useful to calculate the derivative?

<details>
  <summary>Click here for sample answer</summary>
  
  We see that the tangent line at each point ....
</details>

# Integrals

Now that we've given an overview of the concepts of derivatives where infinitesmally small contributions can give us perspectives of a rate; let's look at the reverse process - integrals. How can we rationalize the need for integrals? With derivatives we knew the bulk behavior of a system and wanted to understand the small effects driving changes in the system, so perhaps there are cases where we know the rate or other small effects and need to understand their impact on the system as a whole.

Okay, so with that said, let's consider our previous problem but in reverse i.e. let's do operations on the velocity. If we know there is a series of rates over a set of intervals, we can sum those rates over their respective interval e.g.
\begin{equation}
f(x) = \sum_i^n f'(x_i) \Delta x_i
\end{equation}
or in our problem
\begin{equation}
\text{position}(t) = \text{velocity}(t) \Delta t
\begin{cases} 
1 (t - t_0) & 0 \leq t \leq t_1 \\
0 (t - t_1) & t_1 < t \leq t_2 \\
(t_2 - t) (t - t_2) & t_2 <  t \leq t_3 \\
1 (t - t_3) & t_3 < t
\end{cases}
\end{equation}.
We then can create a similar slider as we did for the derivative where we divide the domain into smaller segments. For our purposes we can do a right Riemann sum which assumes that our curve $f'(x)$ can be approximated by a series of rectangles whose top is centered about a specific point, here we'll pick the top right point as our value to try to keep things simple.

In [99]:
# Function to calculate Riemann rectangles
def riemann_rectangles(velocities, time, num_segments):
    # We are given the number of segments desired, so we'll select that many evenly spaced points in time and velocity
    riemann_times = np.linspace(min(time), max(time), num_segments + 1) # We add 1 to ensure we start at 0 but don't count it

    # We need to duplicate the interior time values so that we have start and endpoints for our rectangles aside from the first and last point
    inner_riemann_times = np.repeat(riemann_times[1:-1], 2) # Repeat each inner element twice

    # Make new riemann times array with dimensions len(inner_riemann_times) + 2 where the 2 serves as the start and end time points 0, t4
    new_riemann_times = np.zeros(len(inner_riemann_times)+2, dtype=float)
    new_riemann_times[1:-1] = inner_riemann_times
    new_riemann_times[-1] = max(time)

    # Find indices for nearest values in time
    idx = find_nearest_index(time, new_riemann_times) # We start at 1 because we want to avoid counting the zero!

    # Make our Riemann rectangles by selecting the values using idx indices
    riemann_rect = velocities[idx[1::2]]

    # Duplicate rectangle heights to coincide with the selected times
    riemann_rect = np.repeat(riemann_rect, 2)

    return riemann_rect, time[idx]

# Function to calculate positions using Riemann sums
def riemann_sum(velocities, time, num_segments):
    # Calculate Riemann rectangles
    rectangles, select_times = riemann_rectangles(velocities, time, num_segments)

    # Select the first and last entries along with only one of the duplicate entries of times and rectangles
    unique_rectangles = rectangles[::2]
    unique_select_times = np.unique(select_times)

    # calculate riemann sums at each point by summing the previous states
    riemann_sums = np.array([0] + [np.sum(unique_rectangles[:i] * (unique_select_times[1:i+1] - unique_select_times[0:i])) for i in range(1, len(unique_rectangles)+1, 1)])

    return riemann_sums, unique_select_times

# Generate positions for different numbers of segments
max_discretization_points = 50
min_discretization_points = 2
list_of_rectangles_and_times = [riemann_rectangles(velocities, time, n) for n in range(min_discretization_points, max_discretization_points + 1)]
list_of_positions_and_times = [riemann_sum(velocities, time, n) for n in range(min_discretization_points, max_discretization_points + 1)]

# Create a figure for the position vs time plot
fig = make_subplots(rows=1, cols=1, subplot_titles=("Position vs Time with Riemann Sum Integration",))

# Add the velocity vs time trace
fig.add_trace(go.Scatter(x=time, y=velocities, mode='lines', name='Velocity'), row=1, col=1)

# Add the initial rectangles trace
fig.add_trace(go.Scatter(x=list_of_rectangles_and_times[0][1], y=list_of_rectangles_and_times[0][0], mode='lines', name='Rectangles'), row=1, col=1)

# Add the initial position vs time trace
fig.add_trace(go.Scatter(x=list_of_positions_and_times[0][1], y=list_of_positions_and_times[0][0], mode='lines', name='Riemann Sum'), row=1, col=1)

# Add the initial rectangles trace
fig.add_trace(go.Scatter(x=time, y=positions, mode='lines', name='True Position'), row=1, col=1)

# Create frames for the slider
frames = [go.Frame(data=[
    go.Scatter(x=list_of_rectangles_and_times[i][1], y=list_of_rectangles_and_times[i][0], mode='lines', name='Riemann'),
    go.Scatter(x=list_of_positions_and_times[i][1], y=list_of_positions_and_times[i][0], mode='lines', name='Position')
], traces=[1, 2], name=str(i)) for i in range(len(list_of_positions_and_times))]

# Add frames to the figure
fig.frames = frames

# Define animation settings for play/pause buttons
animation_settings = dict(frame=dict(duration=20, redraw=True), fromcurrent=True)

# Update layout with buttons and sliders
fig.update_layout(
    updatemenus=[dict(
        type="buttons",  # Define the type of control
        showactive=False,  # Do not show active state
        buttons=[
            dict(label="Play",
                 method="animate",  # Use the animate method
                 args=[None, animation_settings]),  # Settings for the animation
            dict(label="Pause",
                 method="animate",  # Use the animate method
                 args=[[None], dict(frame=dict(duration=0, redraw=False), mode="immediate", transition=dict(duration=0))])
        ]
    )],
    sliders=[{
        'steps': [{'args': [[str(i)], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate'}],
                   'label': str(i+1), 'method': 'animate'} 
                   for i in range(len(list_of_positions_and_times))],  # Define each step of the slider in a list comprehension
        'transition': {'duration': 0},  # No transition duration between steps
        'x': 0.1,  # X position of the slider
        'y': -0.1,  # Y position of the slider
        'currentvalue': {'font': {'size': 20}, 'prefix': 'Discretization points: ', 'visible': True, 'xanchor': 'center'},  # Current value settings
        'len': 0.9  # Length of the slider
    }]
)

# Update axes titles and ranges
fig.update_xaxes(title_text="Time (s)", range=[0, total_time])  # X-axis settings
fig.update_yaxes(title_text="Position and Velocity (units)",
                 range=[min(positions) - 1,
                        max(positions) + 1])  # Y-axis settings

# Set title and layout dimensions
fig.update_layout(title_text="Position vs Time Animation with Riemann Sum Integration", height=600, width=800, showlegend=True)

# Display the figure
pio.show(fig)
